The first thing we have to do arrange the data properly to help train the model. The data can be download from **[here](https://github.com/zo7/painter-by-numbers/releases/tag/data-v1.0)**. It has been preprocessed by some awesome person to reduce size of the images

The downloaded data is consists of 
- **train** : ~75K images 
- **test** : 23817 images

Apart from that we are provided with a file called **train_info.csv** that has the following format: 

**filename,hash(artistname),title,style,genre,data**


102257.jpg,5b39c876740bfc1cfaf544721c43cac3,Uriel,Color Field Painting,abstract,1955.0                                 

So the first thing we have to do is sort all the images into groups of artist hashnames. This can be done in fact with a line of bsshscript
```
cat train_info.csv | tail -n +2 | cut -d, -f2 | sort | uniq | xargs -L 1 mkdir
```

The function performed by each of the following is as follows : 
- **cat** : lists the contents of the csv file
- **tail -n +2** : skips the first line of the csv which are the labels 
- **cut -d, -f2** : returns the 2nd column(given by the flag -f) of the data. -d is used to show the delimeter
- **sort** : sorts the data returned 
- **uniq** : removes duplicate lines. Needs sort before it as it removes only duplicate lines grouped togetner
- **xargs -L 1 mkdir**: takes the input of the previous command one at a a time ( -L 1 ensures that) and attatches it to the mkdire command

And hence this creates the directories with all the hash names

Next we have to move the images into respective artist hashname directories as given in the csv file. I am sure we can do it using python but since I was keen to practice more bash I decided to stick to that. The following command is all that is needed. 

```
cat ../train_info.csv | cut -d, -f1,2 | tr "," " " | xargs -L 1 bash -c 'mv $0 $1' 

```

i am sure there is a much more efficient way to do but I had to scratch my bash itch. The function performed by the commands is as follows : 
- **cat** : lists the contents of the csv file
- **tail -n +2** : skips the first line of the csv which are the labels 
- **cut -d, -f1,2** : returns the 1st and 2nd column(given by the flag -f) of the data. -d is used to show the delimeter.
- **tr "," " "** : replaces the , with a space 
- **uniq** : removes duplicate lines. Needs sort before it as it removes only duplicate lines grouped togetner
- **xargs -L 1 bash -c 'mv $0 $1'**: takes the input of the previous command one at a a time. Moves the image (\$0 : first argument) to the directory ($1 : second argument)


In [2]:
%matplotlib inline

In [4]:
from __future__ import division, print_function

import numpy as np
np.set_printoptions(precision=4, linewidth=100)
import os, json
from glob import glob
from matplotlib import pyplot as plot

In [5]:
import utils; reload(utils)
from utils import plots

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [9]:
path="data/painter-by-numbers/sample/"

In [7]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

## Starting model creation and finetuning

In [10]:
vgg = Vgg16()
batches = vgg.get_batches(path+"train/", batch_size=4)
val_batches = vgg.get_batches(path+"valid/", batch_size=4)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


Exception: When using a generator for validation data, you must specify a value for "nb_val_samples".